In [0]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pickle
import pandas as pd
from IPython.display import Image, display

In [0]:
from tensorflow.keras.preprocessing import image, sequence
from tensorflow.keras.layers import Dense, Convolution2D, Dropout,InputLayer,Add, LSTM,Input, TimeDistributed, Embedding, Bidirectional, Activation, RepeatVector, concatenate, Concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50

In [0]:
vocab_size = 8254

In [5]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


### Model

In [6]:
captions = np.load("/content/drive/My Drive/captions.npy")
next_words = np.load("/content/drive/My Drive/next_words.npy")

print(captions.shape)
print(next_words.shape)

(25493, 40)
(25493, 8254)


In [7]:
images = np.load("/content/drive/My Drive/images.npy")

print(images.shape)

(25493, 2048)


In [8]:
imag = np.load("/content/drive/My Drive/image_names.npy")
        
print(imag.shape)

(25493,)


In [0]:
embedding_size = 128
max_len = 40

In [10]:
image_model = Sequential()

image_model.add(Dense(embedding_size, input_shape=(2048,), activation='relu'))
image_model.add(RepeatVector(max_len))
image_model.summary()

W0713 05:34:58.243446 139746051479424 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               262272    
_________________________________________________________________
repeat_vector (RepeatVector) (None, 40, 128)           0         
Total params: 262,272
Trainable params: 262,272
Non-trainable params: 0
_________________________________________________________________


In [11]:
language_model = Sequential()

language_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=max_len))
language_model.add(LSTM(256, return_sequences=True))
language_model.add(TimeDistributed(Dense(embedding_size)))

language_model.summary()

print(language_model.output.shape)

W0713 05:35:03.425715 139746051479424 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 128)           1056512   
_________________________________________________________________
lstm (LSTM)                  (None, 40, 256)           394240    
_________________________________________________________________
time_distributed (TimeDistri (None, 40, 128)           32896     
Total params: 1,483,648
Trainable params: 1,483,648
Non-trainable params: 0
_________________________________________________________________
(?, 40, 128)


In [0]:

l1 = Concatenate()([image_model.output, language_model.output])
l2 = LSTM(128, return_sequences=True)(l1)
l3 = LSTM(512, return_sequences=False)(l2)
l4 = Dense(vocab_size)(l3)
l5 = Activation('softmax')(l4)

model = Model([image_model.input, language_model.input], l5)
#model.load_weights("./model_weights.h5")
#print(model.summary())



In [0]:
model.load_weights("/content/drive/My Drive/model_weights.h5")
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

In [23]:
hist = model.fit([images, captions], next_words, batch_size=512, epochs=50)

Epoch 1/50
25493/25493 [==============================] - 14s 545us/sample - loss: 1.6082 - acc: 0.5272
Epoch 2/50
25493/25493 [==============================] - 13s 497us/sample - loss: 1.4324 - acc: 0.5689
Epoch 3/50
25493/25493 [==============================] - 13s 504us/sample - loss: 1.3753 - acc: 0.5815
Epoch 4/50
25493/25493 [==============================] - 13s 500us/sample - loss: 1.3405 - acc: 0.5897
Epoch 5/50
25493/25493 [==============================] - 13s 499us/sample - loss: 1.3075 - acc: 0.5951
Epoch 6/50
25493/25493 [==============================] - 13s 497us/sample - loss: 1.2703 - acc: 0.6048
Epoch 7/50
25493/25493 [==============================] - 13s 497us/sample - loss: 1.2356 - acc: 0.6095
Epoch 8/50
25493/25493 [==============================] - 13s 500us/sample - loss: 1.1964 - acc: 0.6215
Epoch 9/50
25493/25493 [==============================] - 13s 497us/sample - loss: 1.1716 - acc: 0.6308
Epoch 10/50
25493/25493 [==============================] - 13s 4

In [0]:
model.save_weights("/content/drive/My Drive/model_weights.h5")

In [0]:
model.save("/content/drive/My Drive/model_Image_Caption.h5")